In [1]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
import configparser
from dateutil.relativedelta import relativedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
end = datetime.date.today()
start = datetime.date(2020, 1, 1)
print('Start date: ' + str(start))
print('End date: ' + str(end))

Start date: 2020-01-01
End date: 2022-03-23


In [3]:
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]
months_in_range

[['2020', '1'],
 ['2020', '2'],
 ['2020', '3'],
 ['2020', '4'],
 ['2020', '5'],
 ['2020', '6'],
 ['2020', '7'],
 ['2020', '8'],
 ['2020', '9'],
 ['2020', '10'],
 ['2020', '11'],
 ['2020', '12'],
 ['2021', '1'],
 ['2021', '2'],
 ['2021', '3'],
 ['2021', '4'],
 ['2021', '5'],
 ['2021', '6'],
 ['2021', '7'],
 ['2021', '8'],
 ['2021', '9'],
 ['2021', '10'],
 ['2021', '11'],
 ['2021', '12'],
 ['2022', '1'],
 ['2022', '2'],
 ['2022', '3']]

In [4]:
#configs = configparser.ConfigParser()
#configs.read('config.ini')
#YOUR_API_KEY = configs['NYT']['aaLRz4dlIAdYkcaKeoADrUjmgeaU9Omy']

In [5]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + '4irjbJ6eLQseGkjqjLg8lFwpw1haGGNN'
    response = requests.get(url).json()
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'snippet': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'snippet' in article:
                data['snippet'].append(article['snippet'])
            else:
                data['snippet'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['web_url'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        response = send_request(date)
        df = parse_response(response)
        total += len(df)
        df.to_csv('headlines/' + date[0] + '-' + date[1] + '.csv', index=False)
        print('Saving headlines/' + date[0] + '-' + date[1] + '.csv...')
    print('Number of articles collected: ' + str(total))

In [8]:
get_data(months_in_range)

Date range: ['2020', '1'] to ['2022', '3']
Saving headlines/2020-1.csv...
Saving headlines/2020-2.csv...
Saving headlines/2020-3.csv...
Saving headlines/2020-4.csv...
Saving headlines/2020-5.csv...
Saving headlines/2020-6.csv...
Saving headlines/2020-7.csv...
Saving headlines/2020-8.csv...
Saving headlines/2020-9.csv...
Saving headlines/2020-10.csv...
Saving headlines/2020-11.csv...
Saving headlines/2020-12.csv...
Saving headlines/2021-1.csv...
Saving headlines/2021-2.csv...
Saving headlines/2021-3.csv...
Saving headlines/2021-4.csv...
Saving headlines/2021-5.csv...
Saving headlines/2021-6.csv...
Saving headlines/2021-7.csv...
Saving headlines/2021-8.csv...
Saving headlines/2021-9.csv...
Saving headlines/2021-10.csv...
Saving headlines/2021-11.csv...
Saving headlines/2021-12.csv...
Saving headlines/2022-1.csv...
Saving headlines/2022-2.csv...
Saving headlines/2022-3.csv...
Number of articles collected: 120850
